## Graph and Session

In [ ]:
# Import tensorflow
import tensorflow as tf

In [ ]:
# Tensorflow support Data Flow Graphs

# phase 1: assemble a graph
a = tf.add(2, 3) # tf automatically name the nodes when you don't explicitly name them
# what is node? Nodes: operators, variables, and constants
# What is edges: tensors (data)
print a # Not 5, so how to get the value of a? Create a session, assign it to variable sess so can call it later
        # within the session, evaluate the graph to fetch the value of a (phase 2)

# phase 2: use a session to execute operations in the graph
sess = tf.Session()  # a Session object encapsulate the environment in which Operation objects are executed, and the Tensor objects are evaluated
print sess.run(a) # output the result 5
sess.close # free memory

In [ ]:
# More graph
x = 2
y = 3
op1 = tf.add(x, y) # 5
op2 = tf.multiply(x, y) # 6
op3 = tf.pow(op2, op1) # 6 ^ 5
with tf.Session() as sess:
    op3 = sess.run(op3)

In [ ]:
# It is possible to break graphs into several chunks and run them parallelly across multiple CPUs, GPUs, or devices
# Distributed Computation
# To put part of a graph on a specific CPU or GPU
# Create a graph
with tf.device('/cpu:2'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b) # Multiplies matrix a by matrix b, producing a * b, (a, b must be matrix, changed in TF 1.1 version)
    
# Creates a session with log_device_placement set to True
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Runs the op.
print sess.run(c)

In [ ]:
# if want more than one graph
# multiple graphs require multiple sessions, each will try to use all available resources by default
# can't pass data btw them without passing them through python/numpy. which does not work in distributed
# better to have disconnected subgraphs within one graph

g = tf.Graph() # create a graph
with g.as_default(): # add operators to a graph, set it as default
    x = tf.add(3, 5)
    
with tf.Session(graph=g) as sess: # session is run on the graph g; session would be close automatically
    print sess.run(x)

In [ ]:
# Do not mix default graph and user created graphs
g1 = tf.get_default_graph()
g2 = tf.Graph()

with g1.as_default():
    a = tf.constant(3)
    
with g2.as_default():
    b = tf.constant(5)

sess = tf.Session() # run on the default graph
print sess.run(a) # error if run(b) because default graph does not have node b

with tf.Session(graph=g2) as sess:
    print sess.run(b)


#### Why graphs?
- save computation, only run subgraphs that lead to the values we want to fetch
- Break computation into small, different pieces to facilitates auto-differention
- Facilitate distributed computation, spread the work across multiple CPUs, GPUs, or devices
- Many common machine learning models are commonly taught and visualized as directed graphs already

## TensorBoard

In [3]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
with tf.Session() as sess:
    # To activate TensorBoard on this program, add a line after we've built the graph, right before running the train loop
    writer = tf.summary.FileWriter('./graphs', sess.graph)  # './graphs' is the logs_dir
    print sess.run(x)

writer.close() # close the writer when you'are done using it

# Next, go to Terminal, run the program. Make sure that your present working directory is the same as where you ran your Python code.
# python  [ yourprogram . py ]
# tensorboard  --logdir = "./graphs"
# Open your browser and go to http://localhost:6006/  (or the link you get back after running tensorboard command).


5


## Tensorflow Constant types

In [ ]:
# create constants of scalar or tensor values
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

a = tf.constant([2, 2], name='vaector')
b = tf.constant([[1, 2], [3, 4]], name='matrix')

tf.zeros(shape, dtype=tf.float32, name=None)  # like numpy.zeros
tf.zeros([2, 3], tf.int32)

# like numpy.zeros_like, create a tensor of shape and type (unless type is specified) as the input_tensor but all elements are zeros.
tf.zeros_like(tensor, dtype=None, name=None, optimize=True) # optimize: if true, attempt to statically determine the shape of 'tensor' and encode it as a constant.

tf.ones(shape, dtype=tf.float32, name=None)  # like numpy.ones
tf.ones_like(tensor, dtype=None, name=None, optimize=True) # like numpy.ones_like

tf.fill(dims, value, name=None) # create a tensor filled with a scalar value.

## Create constants that are sequences
# Create a sequence of num evenly-spaced values are generated beginning at start. The values in the sequence increase by
# (stop - start) / (num - 1). The last one is exactly stop
tf.linspace(start, stop, num, name=None)

# create a sequence of numbers that begins at start and extends by increments of delta up to but not including limit
tf.range(start, limit=None, delta=1, dtype=None, name='range')

## Tensorflow sequences are not iterable
for  _  in  tf.linspace(0, 10, 4):   # TypeError("'Tensor' object is not iterable.")
for  _  in  tf.range(4):   # TypeError("'Tensor' object is not iterable.")

## Also could generate random constants from certain dist

tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
# the values whose magnitude is more than 2 sd from the mean are dropped and re-picked
tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)
# randomly shuffle a tensor along its first dimension
tf.random_shuffle(value, seed=None, name=None)
# Slices a shape size portion out of value at a uniformly chosen offset. Requires value.shape >= size.
tf.random_crop(value, size, seed=None, name=None)
# Draws samples from a multinomial distribution. logits: 2-D Tensor with shape [batch_size, num_classes]. Each slice [i, :] represents the unnormalized log probabilities for all classes.
tf.multinomial(logits, num_samples, seed=None, name=None) 
tf.random_gamma(shape, alpha, beta=None, dtype=tf.float32, seed=None, name=None)
tf.set_random_seed(seed) # set the graph-level random seed

## Math Ops

In [ ]:
# Element-wise mathematical operations
tf.add(x, y, name=None) # x, y: a tensor. Add supports broadcasting. AddN does not. 
tf.substract(x, y, name=None) # x, y: a tensor. supports broadcasting
tf.multiply(x, y, name=None) # x, y: a tensor. supports broadcasting
tf.scalar_mul(scalar, x) # x: A Tensor or IndexedSlices to be scaled.
tf.div(x, y, name=None) # using Python2 division operator semantics. That is, if one of x or y is a float, then the result will be a float. Otherwise, the output will be an integer type. Prefer using the Tensor division operator or tf.divide which obey Python division operator semantics.using Python 2 division operator semantics. 
tf.divide(x, y, name=None) # python style division of x by y
tf.truediv(x, y, name=None) # Python3 division operator semantics where all integer arguments are cast to floating types first.
tf.floordiv(x, y, name=None) # Divides x / y elementwise, rounding toward the most negative integer.
tf.realdiv(x, y, name=None) # If x and y are reals, this will return the floating-point division.
tf.floor_div(x, y, name=None) # Returns x//y element-wise
tf.truncateddiv(x, y, name=None) # Truncation designates that negative numbers will round fractional quantities toward zero. I.e. -7 / 5 = -1.
tf.truncatemod(x, y, name=None) # reminder of division. C semantics in that the result here is consistent with a flooring divide. E.g. floor(x / y) * y + mod(x, y) = x.
tf.floormod(x, y, name=None) or tf.floormod  # remainder of division. When x < 0 xor y < 0 is true, this follows Python semantics in that the result here is consistent with a flooring divide. E.g. floor(x / y) * y + mod(x, y) = x.
...

# Array Operations
tf.add_n(inputs, name=None) # inputs: A list of Tensor objects, each with same shape and type.
tf.Concat, Slice, Split, Constant, Rank, Shape, Shuffle
...

# Matrix Operations
MatMul, MatrixInverse, MatrixDeterminant,...

# Stateful Operations
Variable, Assign, AssignAdd, ...

# Neural Network Building Blocks
Softmax, Sigmoid, ReLU, Convolution2D, MaxPool, ...

# Checkpointing Operations
Save, Restore

# Queue and synchonization operations
Enquene, Dequene, MutexAcquire, MutexRelease, ...

# Control flow operations
Merge, Switch, Enter, Leave, NextIteration

## Data Types

In [ ]:
# TensorFlow takes in Python native types such as Python boolean values, numeric values (integers, floats), and strings. 
# Single values will be converted to 0-d tensors (or scalars), lists of values will be converted to 1-d tensors (vectors), 
# lists of lists of values will be converted to 2-d tensors (matrices), and so on. 

# https://www.tensorflow.org/programmers_guide/dims_types

# NumPy Data Types
# Could pass NumPy types to TensorFlow
# Most of the times, you can use TensorFlow types and NumPy types interchangeably.
tf.Session.Run(tf.ones([2, 2], np.float32))  ==>   numpy array [[1.0  1.0 ], [ 1.0   1.0 ]]

## Variables

In [ ]:
# A constant's value is stored in the graph and its value is replicated wherever the graph is loaded. 
# A variable is stored separately, and may live on a parameter server.

# Declare variables
# create variable a with scalar value
a = tf.Variable(2, name="scalar")
# create variable b as a vector
b = tf.Variable([2, 3], name="vector")
# create variable c as a 2x2 matrix
c = tf.Variable([[0 ,  1], [2, 3]], name="matrix")
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784, 10]))

# tf.Variable holds several ops:
x.initializer # init 
x.value() # read op 
x.assign(...) # write op 
x.assign_add(...)
# and more

# You have to initialize variables before using them.  If you try to evaluate the variables before initializing them 
# you’ll run into FailedPreconditionError: Attempting to use uninitialized value tensor.

# The easiest way is initializing all variables at once using: tf.global_variables_initializer()

init = tf.global_variables_initializer()
with tf.Session() as sess: 
    tf.run(init)
    
# To initialize only a subset of variables, you use tf.variables_initializer() 
# with a list of variables you want to initialize:

init_ab = tf.variables_initializer([a, b], name="init_ab")
with tf.Session() as sess:
    tf.run(init_ab)

# You can also initialize each variable separately using tf.Variable.initializer
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784, 10]))
with tf.Session() as sess:
    tf.run(W.initializer)
    
    
"""Evaluate values of variables"""
# If we print the initialized variable, we only see the tensor object.
# To get the value of a variable, we need to evaluate it using eval()
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print W.eval()
    
# Assign values to variables
# We can assign a value to a variable using tf.Variable.assign()
W = tf.Variable(10)
W.assign(100) # W.assign(100) doesn’t assign the value 100 to W, but instead create an assign op to do that. For this op to take effect, we have to run this op in session.
with tf.Session() as sess:
    sess.run(W.initializer)
    print W.eval() # >> 10
    
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(assign_op) # don't have to init W as init ops is actually a assign ops, assigning the init value
    print W.eval() # >> 100

# create a variable whose original value is 2
a = tf.Variable(2, name="scalar")
# assign a * 2 to a and call that op a_times_two
a_times_two = a.assign(a * 2)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
# have to initialize a, because a_times_two op depends on the value of a 
sess.run(a_times_two) # >> 4
sess.run(a_times_two) # >> 8
sess.run(a_times_two) # >> 16

# For simple incrementing and decrementing of variables, TensorFlow includes the 
# tf.Variable.assign_add() and tf.Variable.assign_sub() methods. 
# Unlike tf.Variable.assign(), tf.Variable.assign_add() and tf.Variable.assign_sub() 
# don’t initialize your variables for you because these ops depend on the initial values of the variable.
W = tf.Variable(10)
with tf.Session() as sess:
    sess.run(W.initializer)
    print sess.run(W.assign_add(10)) # >> 20 
    print sess.run(W.assign_sub(2)) # >> 18
    
# Because TensorFlow sessions maintain values separately, each Session can have its own current value for a variable defined in a graph.
W = tf.Variable(10)
sess1 = tf.Session() sess2 = tf.Session()
sess1.run(W.initializer) sess2.run(W.initializer)
print sess1.run(W.assign_add(10)) # >> 20 print sess2.run(W.assign_sub(2)) # >> 8
print sess1.run(W.assign_add(100)) # >> 120 print sess2.run(W.assign_sub(50)) # >> -42
sess1.close()
sess2.close()

# can declare a variable that depends on other variables.
W = tf.Variable(tf.truncated_normal([700, 10]))
U = tf.Variable(W * 2) #  should use initialized_value() to make sure that W is initialized before its value is used to initialize W.
U = tf.Variable(W.intialized_value() * 2) # Not sure whether it's a good idea

## InteractiveSession

In [ ]:
# The only difference is an InteractiveSession makes itself the default session so you can call run() or eval() without explicitly call the session.
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
# We can just use 'c.eval()' without passing 'sess'
print(c.eval()) 
sess.close()

# tf.get_default_session() returns the default session for the current thread. The returned Session will be the innermost session 
# on which a Session or Session.as_default() context has been entered.

## Placeholders and feed_dict

In [ ]:
# With the graph assembled, we, or our clients, can later supply their own data when they need to execute the computation.
# Define a placeholder
# Shape specifies the shape of the tensor that can be accepted as actual value for the placeholder. 
# shape=None means that tensors of any shape will be accepted. Using shape=None is easy to construct graphs, 
# but nightmarish for debugging. You should always define the shape of your placeholders as detailed as possible.
tf.placeholder(dtype, shape=None, name=None)

# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape =[3])
# create a constant of type float 32-bit, shape is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)
# use the placeholder as you would a constant or a variable
c = a + b  # Short for tf.add(a, b)
"""If  we  try  to fetch c ,  we will run  into  error."""
with tf.Session() as sess:
    print(sess.run(c))
>>   NameError # because to compute c, we need the value of a, but a is just a placeholder without actual value. We have to first feed actual value into a.


with tf.Session() as sess:
    # feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]}
    # fetch value of c
    print(sess.run(c, {a: [1, 2, 3]}))

# We can feed as any data points to the placeholder as we want by iterating through the data set and feed in the value one at a time.
with tf.Session() as sess:
    for a_value in list_of_a_values:
        print (sess.run(c, {a: a_value}))
        
# You can feed values to tensors that aren’t placeholders. Any tensors that are feedable can be fed. To check if a tensor is feedable or not, use:
tf.Graph.is_feedable(tensor)

# create Operations, Tensors, etc (using the default graph)
a = tf.add(2, 5)
b = tf.mul(a, 3)
# start up a `Session` using the default graph 
sess = tf.Session()
# define a dictionary that says to replace the value of `a` with 15 
replace_dict = {a: 15}
# Run the session, passing in `replace_dict` as the value to `feed_dict` 
sess.run(b, feed_dict=replace_dict) # returns 45

## The trap of lazy loading

In [ ]:
#  Lazy loading is a term that refers to a programming pattern when you defer declaring/initializing an object until it is loaded. In the context of
# TensorFlow, it means you defer creating an op until you need to compute it.
# correct one
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(z)
    writer.close()

# not good one
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(tf.add(x, y)) # create the op add only when you need to compute it 
    writer.close()
        
        